In [2]:
from statsbombpy import sb #Manage stats bomb data
import pandas as pd #Work with any type of data
from mplsoccer import Pitch,Sbopen #Visualize soccer data
import matplotlib.pyplot as plt #Math graphs and stuff like that. Not sure if needed
import numpy as np #Work with matrices and number precisly
import networkx as nx

MexVSArgID= 3857289
MexVsArabID = 3857260
worldCupID = 43
seasonID = 106

parser = Sbopen()
df,related,freezer,tactics = parser.event(MexVSArgID) #Get info about the match (temach xdddd)

df = df[df['team_name']=='Mexico'] #Data from Mexico only
passes = df[df['type_name']=='Pass'] #Passes from Mexico
filteredPasses = passes[['player_name','x','y','end_x', 'end_y','pass_recipient_name','outcome_id','outcome_name']] #Data I need from the passes
successfullPasses = filteredPasses[filteredPasses['outcome_name'].isnull()] #Only successful passes

#Herramienta sorpresa que nos ayudará más tarde
playersName = successfullPasses['player_name'].unique().tolist() 
playersID = {}

for index  in range(len(playersName)):
    playersID[playersName[index]] = index

successfullPasses = successfullPasses.reset_index(drop=True) #New 0 to n id
# for debugging print(successfullPasses.shape)

print(playersID)

{'José Andrés Guardado Hernández': 0, 'Jesús Daniel Gallardo Vasconcelos': 1, 'Héctor Alfredo Moreno Herrera': 2, 'Héctor Miguel Herrera López': 3, 'Néstor Alejandro Araújo Razo': 4, 'Kevin Nahin Álvarez Campos': 5, 'Ernesto Alexis Vega Rojas': 6, 'Luis Gerardo Chávez Magallón': 7, 'Francisco Guillermo Ochoa Magaña': 8, 'César Jasib Montes Castro': 9, 'Hirving Rodrigo Lozano Bahena': 10, 'Érick Gabriel Gutiérrez Galaviz': 11, 'Raúl Alonso Jiménez Rodríguez': 12, 'Carlos Uriel Antuna Romero': 13, 'Roberto Carlos Alvarado Hernández': 14}


In [3]:
#Prepares the adjc matrix
adjcMatrix = []
playersMexico = 15

for _ in range(playersMexico): #You use _ when you don't care about the index value
    adjcMatrix.append([])

for i in range(playersMexico): #i goes from 0 to 14 (playersMexico - 1)
    for _ in range(playersMexico):
        adjcMatrix[i].append(0)

#Iterates over each row in successfulPasses
for index, row in successfullPasses.iterrows():
    passer = row['player_name']  
    receiver = row['pass_recipient_name']
    adjcMatrix[playersID[passer]][playersID[receiver]] += 1
    start_pos = (row['x'], row['y'])  
    end_pos = (row['end_x'], row['end_y'])
    #print(f"Pase de {index} {passer} a {receiver} desde {start_pos} hasta {end_pos} xd")

In [4]:
for player in adjcMatrix:
    print(player)

sum = 0
for passer in adjcMatrix:
    for passed in passer:
        sum += passed
print(sum)
#Btw, Héctor Herrera is in index 3. You confused him with Héctor Moreno

[0, 2, 4, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]
[3, 0, 6, 1, 0, 0, 3, 1, 1, 2, 0, 1, 0, 0, 1]
[2, 6, 0, 3, 0, 1, 0, 0, 5, 7, 2, 8, 2, 0, 2]
[2, 1, 1, 0, 10, 1, 0, 6, 2, 7, 1, 3, 1, 2, 0]
[0, 0, 1, 5, 0, 6, 1, 5, 1, 19, 2, 0, 5, 2, 0]
[0, 0, 0, 0, 6, 0, 2, 5, 0, 0, 2, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0]
[1, 0, 1, 2, 9, 3, 0, 0, 0, 1, 1, 1, 0, 0, 2]
[2, 1, 6, 1, 0, 0, 0, 2, 0, 8, 0, 0, 0, 0, 0]
[0, 1, 11, 6, 11, 0, 0, 0, 8, 0, 0, 8, 1, 2, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0]
[0, 3, 5, 6, 1, 0, 0, 0, 0, 4, 1, 0, 0, 0, 5]
[0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0]
304


In [5]:
# Cell 4
# Convierte la matriz de adyacencia a una matriz numpy
adj_matrix_np = np.array(adjcMatrix)

# Calcula el vector propio y el valor propio de la matriz de adyacencia
eigenvalues, eigenvectors = np.linalg.eig(adj_matrix_np)

# Obtiene el índice del mayor valor propio
max_eigenvalue_index = np.argmax(eigenvalues)

# Obtiene el vector propio correspondiente al mayor valor propio
principal_eigenvector = eigenvectors[:, max_eigenvalue_index]

# Normaliza el vector propio
centrality = principal_eigenvector / np.sum(principal_eigenvector)

# Imprime la centralidad para cada jugador
for player, centrality_value in zip(playersID.keys(), centrality):
    print(f"{player}: {centrality_value:.4f}")

José Andrés Guardado Hernández: 0.0323-0.0000j
Jesús Daniel Gallardo Vasconcelos: 0.0494-0.0000j
Héctor Alfredo Moreno Herrera: 0.1060-0.0000j
Héctor Miguel Herrera López: 0.1360-0.0000j
Néstor Alejandro Araújo Razo: 0.1662-0.0000j
Kevin Nahin Álvarez Campos: 0.0470-0.0000j
Ernesto Alexis Vega Rojas: 0.0021-0.0000j
Luis Gerardo Chávez Magallón: 0.0781-0.0000j
Francisco Guillermo Ochoa Magaña: 0.0815-0.0000j
César Jasib Montes Castro: 0.1742-0.0000j
Hirving Rodrigo Lozano Bahena: 0.0070-0.0000j
Érick Gabriel Gutiérrez Galaviz: 0.0813-0.0000j
Raúl Alonso Jiménez Rodríguez: 0.0168-0.0000j
Carlos Uriel Antuna Romero: 0.0087-0.0000j
Roberto Carlos Alvarado Hernández: 0.0133-0.0000j
